In [70]:
import argparse
from typing import Dict, Tuple, List, Literal
import logging
import copy
import pandas as pd

import numpy as np
import scipy.stats

import molmass
from pyteomics import mzml as pytmzml

from draft import *
import formulaUtils

In [71]:
mzml_1 = "/Users/gkreder/Downloads/2024-07-15_mschemutils-refactor/Pos_01.mzML"
mzml_2 = "/Users/gkreder/Downloads/2024-07-15_mschemutils-refactor/Pos_05.mzML"
starting_index = 1
index_1 = 750
index_2 = 633
quasi_x = 114.5
quasi_y = -0.238
R = 10000
match_acc = 100.0 / R
res_clearance = 200.0 / R
subformula_tolerance = 100.0 / R
du_min = -0.5
min_spectrum_quasi_sum = 20
min_total_peaks = 2

abs_cutoff = None
rel_cutoff = None
QUASI_CUTOFF_DEFAULT = 5
quasi_cutoff = QUASI_CUTOFF_DEFAULT
exclude_peaks = None
pdpl = None

parent_mz = 271.0601
parent_formula = molmass.Formula("C15H11O5").formula
out_prefix = "Genistein_v_DMG_0V-Pos_01_750_v_Pos_05_633"


gain_control = False

In [72]:

def H(x):
    h = -1 * np.sum( x * ( np.log(x) ) )
    return(h)

def calc_G2(a_i, b_i, S_A, S_B, M):
        t1 = ( a_i + b_i ) * np.log( ( a_i + b_i ) / ( S_A + S_B ) ).apply(lambda x : 0 if np.isinf(x) else x)
        t2 = a_i * np.log(a_i / S_A).apply(lambda x : 0 if np.isinf(x) else x)
        t3 = b_i * np.log(b_i / S_B).apply(lambda x : 0 if np.isinf(x) else x)
        G2 = -2 * (t1 - t2 - t3).sum()
        pval_G2 = scipy.stats.chi2.sf(G2, df = M - 1)
        return(G2, pval_G2)

def calc_D2(a_i, b_i, S_A, S_B, M):
        num = np.power( ( S_B * a_i )  - ( S_A * b_i ) , 2) 
        denom = a_i + b_i
        D2 = np.sum(num  / denom) / (S_A * S_B)
        
        pval_D2 = scipy.stats.chi2.sf(D2, df = M - 1)
        return(D2, pval_D2)

def calc_SR(a_i, b_i, S_A, S_B):
    num = ( S_B * a_i ) - ( S_A * b_i )
    denom = np.sqrt(S_A * ( S_A + S_B ) * ( a_i + b_i ))
    SR_ai = num / denom

    num = ( S_A * b_i ) - ( S_B * a_i )
    denom = np.sqrt(S_B * ( S_A + S_B ) * ( a_i + b_i ))
    SR_bi = num / denom

    return((SR_ai, SR_bi))

# Cosine Distance
def sqF(x):
    return(np.sqrt( np.power( x, 2 ).sum() ) )

def cosine_distance(p_Ai, p_Bi):
    num = p_Ai * p_Bi.sum()
    denom = sqF(p_Ai) * sqF(p_Bi)
    csd = num / denom
    return csd

def jensen_shanon_distance(p_Ai, p_Bi, H_pA, H_pB):
    jsd = H( ( 0.5 * ( p_Ai.fillna(0.0) + p_Bi.fillna(0.0) ) ) ) - ( 0.5 * H_pA ) - ( 0.5 *  H_pB  )
    return jsd

##### STOPPED HERE GKREDER

In [73]:
spectra = get_spectra_by_indices([mzml_1, mzml_2], [index_1 - starting_index, index_2 - starting_index], gain_control)
validate_spectrum_pair(spectra)
kw = {'abs_cutoff' : abs_cutoff, 
      'quasi_x' : quasi_x,
      'quasi_y' : quasi_y,
      'rel_cutoff' : rel_cutoff,
      'quasi_cutoff' : quasi_cutoff,
      'pdpl' : pdpl,
      'exclude_peaks' : exclude_peaks,
      'match_acc' : match_acc,
      'parent_mz' : parent_mz,
      'res_clearance' : res_clearance,
      'sort_intensity' : True
      }
spectra_filtered = [filter_and_convert_spectrum_complete(spectrum, **kw) for spectrum in spectra]
gray_kw = {
    'quasi_x' : quasi_x,
    'quasi_y' : quasi_y,
    'parent_mz' : parent_mz,
    'match_acc' : match_acc
}
gray_spectra = [filter_and_convert_spectrum_complete(spectrum, **gray_kw) for spectrum in spectra]

In [74]:
def merge_spectra(s1 : Dict, s2 : Dict, tolerance : float,
                  suffixes : List[str] = ["A", "B"],
                  direction : str = 'nearest',
                  keys : Dict[str, str] = {'m/z array' : 'm/z',
                                           'intensity array' : 'intensity',
                                           'quasi array' : 'quasi'},
                join_key : str = 'm/z array') -> pd.DataFrame:
    """Takes two spectra and merges them into a single DataFrame based on the join_key and match tolerance.
    Spectra are sorted by join_key before merging."""
    # Add extra join key column for merging
    jk = f"{join_key}_join"
    dfs = [pd.DataFrame(dict({v : s[k] for k, v in keys.items()}, **{jk : s[join_key]})) for s in [s1, s2]]
    dfs = [df.sort_values(jk) for df in dfs]
    df = pd.merge_asof(dfs[0], dfs[1],
                       tolerance = tolerance,
                       on = jk,
                       suffixes = [f'_{x}' for x in suffixes],
                       direction = direction).drop(columns = jk)
    cols_sorted = [f"{k}_{x}" for k in keys.values() for x in suffixes]
    return pd.DataFrame(df[cols_sorted])


In [75]:
def calc_join_formulas(mz_1 : float, mz_2 : float,
                      tolerance : float,
                      du_min : float,
                      possible_formulas : List[Tuple[str, float]]) -> Tuple[List[str], List[float], List[str]]:
    """Given two m/z values, calculates the possible formulas and their expected m/z's."""
    if np.isnan(mz_1) and np.isnan(mz_2):
        raise ValueError("Both m/z values are NaN, at least one must be valid.")
    best_formulas, th_masses, errors = formulaUtils.findBestForms(np.nanmean((mz_1, mz_2)),
                                                             possible_formulas,
                                                             toleranceDa = tolerance,
                                                             DuMin=du_min,)
    if best_formulas[0] is None:
        return None, None, None
    else:
        return best_formulas, th_masses, errors

def calc_formulas(merged_spectrum : pd.DataFrame, parent_formula : str,
                  mz_col : str = 'm/z',
                  suffixes : List[str] = ['A', 'B'],) -> Tuple[List[List[str]], List[List[float]]]:
    """Calculates the formulas and expected m/z's for a merged spectrum."""
    all_formulas = formulaUtils.generateAllForms(parent_formula)
    # Zip the m/z values across the spectra
    z = zip(*merged_spectrum[[f"{mz_col}_{s}" for s in suffixes]].values.T)
    formula_finds = [calc_join_formulas(mz_a, mz_b, subformula_tolerance, du_min, all_formulas) for mz_a, mz_b in z]
    best_formulas, th_masses, errors = zip(*formula_finds)
    best_formulas_str = [", ".join([x.replace("None", "") for x in f]) if f is not None else None for f in best_formulas]
    th_masses_str = [", ".join([str(x) for x in f]) if f is not None else None for f in th_masses]
    return best_formulas_str, th_masses_str


In [76]:
merged_spectrum = merge_spectra(spectra_filtered[0], spectra_filtered[1], match_acc)
best_formulas_str, th_masses_str = calc_formulas(merged_spectrum, parent_formula)
merged_spectrum['formula'] = best_formulas_str
merged_spectrum['m/z_calculated'] = th_masses_str
# If there's no pre-defined peak list (pdpl) and a parent formula was specified drop rows with no formula
if pdpl is None and parent_formula is not None:
    formula_spectrum = merged_spectrum.dropna(subset = ['formula'])
else:
    formula_spectrum = merged_spectrum.copy()

In [129]:
def calc_row_metrics(row : pd.Series, quasi_sums : List[float], M : int, join_type : Literal['Union', 'Intersection'],
                     suffixes : List[str] = ['A', 'B']):
        quasi_intensities = [row[f"quasi_{s}"] for s in suffixes] # a_i and b_i
        if np.isnan(quasi_intensities).any() and (join_type == 'Intersection'):
            return None
        quasi_intensities_series = [pd.Series([x]) for x in quasi_intensities]
        D2, _ = calc_D2(*[x.fillna(0.0) for x in quasi_intensities_series],
                         *quasi_sums,
                         M)
        G2, _ = calc_G2(*[x.fillna(0.0) for x in quasi_intensities_series],
                         *quasi_sums,
                         M)
         # Calculate Standardized Residual (SR)
        SRs = calc_SR(*[x.fillna(0.0) for x in quasi_intensities_series],
                               *quasi_sums)
        SRs = [x.values[0] for x in SRs]
        return dict({f'D^2_{join_type}' : D2, f'G^2_{join_type}' : G2}, **{f'SR_{s}_{join_type}' : SR for s, SR in zip(suffixes, SRs)})
      

def calc_join_metrics(merged_spectrum : pd.DataFrame, join_type : Literal['Union', 'Intersection'],
               suffixes : List[str] = ['A', 'B'],
               keys : Dict[str, str] = {'quasi' : '(quasicounts)',
                                        'intensity' : '(raw)',},
               ):
    """Calculates the comparison metrics for two merged spectra given a join type (intersection or union)"""
    # Create a temporary DataFrame on which to calculate comparison statistics
    if join_type == "Union":
        dfC = merged_spectrum.copy()
    elif join_type == "Intersection":
        drop_subset = [f"m/z_{s}" for s in suffixes]
        dfC = merged_spectrum.dropna(subset = drop_subset).copy()

    metrics = {f"S_{s} {keys[k]}" : merged_spectrum[f"{k}_{s}"].sum() for s in suffixes for k in keys.keys()}
    metrics['M'] = dfC.shape[0]

    

    quasi_intensities = [dfC[f"quasi_{s}"] for s in suffixes] # a_i and b_i
    quasi_sums = [dfC[f"quasi_{s}"].sum() for s in suffixes] # S_A and S_B


    # Calculate D^2
    D2, pval_D2 = calc_D2(*[x.fillna(value = 0.0) for x in quasi_intensities],
                          *quasi_sums,
                          metrics['M']) # a_i, b_i, S_A, S_B, M
    metrics['D^2'] = D2
    metrics['pval_D^2'] = pval_D2


    # Calculate G^2
    G2, pval_G2 = calc_G2(*[x.fillna(value = 0.0) for x in quasi_intensities],
                          *quasi_sums,
                          metrics['M']) # a_i, b_i, S_A, S_B, M
    metrics['G^2'] = G2
    metrics['pval_G^2'] = pval_G2


    ps = [quasi_intensities[i] / quasi_sums[i] for i in range(len(suffixes))] # p_Ai and p_Bi


    entropies = [H(p) for p in ps] # H_A and H_B
    metrics.update({f"Entropy_{s}" : e for s, e in zip(suffixes, entropies)})
    perplexities = [np.exp(h) for h in entropies] # Perplexity_A and Perplexity_B
    metrics.update({f"Perplexity_{s}" : p for s, p in zip(suffixes, perplexities)})


    # Jensen-Shannon divergence
    jsd = jensen_shanon_distance(*ps, *entropies)
    metrics['JSD'] = jsd

    # Cosine distance
    csd = cosine_distance(*ps)
    metrics['Cosine Similarity'] = csd


    dfC[f"D^2_{join_type}"] = None
    dfC[f"G^2_{join_type}"] = None
    dfC[f"SR_A_{join_type}"] = None
    dfC[f"SR_B_{join_type}"] = None


    for i_row, row in dfC.iterrows():
         row_res = calc_row_metrics(row, quasi_sums, metrics['M'], join_type)
         if row_res is not None:
            for k, v in row_res.items():
                dfC.at[i_row, k] = v
    return metrics, dfC

def calc_spectra_metrics(merged_spectrum : pd.DataFrame,
                         suffixes : List[str] = ['A', 'B'],
                         keys : Dict[str, str] = {'quasi' : '(quasicounts)',
                                                  'intensity' : '(raw)',},
                         ) -> Dict:
    """Calculates the comparison metrics for two merged spectra across join types (intersection and union)"""
    metrics = {}
    for join_type in ['Union', 'Intersection']:
        metrics_j, dfC = calc_join_metrics(merged_spectrum=merged_spectrum,
                                           join_type=join_type,
                                           suffixes=suffixes,
                                           keys=keys)
        metrics[join_type] = {}
        metrics[join_type]['metrics'] = metrics_j
        metrics[join_type]['df'] = dfC
    return metrics
    
# calc_join_metrics(formula_spectrum, join_type='Intersection', suffixes=['A', 'B'])
metrics = calc_spectra_metrics(formula_spectrum)

/Users/gkreder/MSChemUtils/.pixi/envs/default/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/gkreder/MSChemUtils/.pixi/envs/default/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [130]:
metrics['Intersection']


{'metrics': {'S_A (quasicounts)': 7542.892182838021,
  'S_A (raw)': 227791.36415100098,
  'S_B (quasicounts)': 4851.5105403652,
  'S_B (raw)': 146475.21209716797,
  'M': 4,
  'D^2': 1.1937393952537017,
  'pval_D^2': 0.7545062516866095,
  'G^2': 1.237690940773298,
  'pval_G^2': 0.7439793785665907,
  'Entropy_A': 0.04404155842149113,
  'Entropy_B': 0.03592095163404346,
  'Perplexity_A': 1.0450257836080072,
  'Perplexity_B': 1.0365739037789607,
  'JSD': 5.3187753917625924e-05,
  'Cosine Similarity': 1    0.001953
  2    0.002886
  3    0.001397
  4    1.004904
  Name: quasi_A, dtype: float64},
 'df':         m/z_A       m/z_B    intensity_A    intensity_B      quasi_A  \
 1  153.018686  153.015881     503.410156     201.177521    14.557462   
 2  215.065487  215.074365     685.942200     392.497070    21.509676   
 3  243.066303  243.065204     322.632690     173.600006    10.416106   
 4  271.060642  271.060542  226059.375000  145707.937500  7490.086856   
 
        quasi_B   formula   m

In [88]:
formula_spectrum

,m/z_A,m/z_B,intensity_A,intensity_B,quasi_A,quasi_B,formula,m/z_calculated
0,149.023861,NaN,220.004105,NaN,6.322083,NaN,C8H5O3,149.02386901986
1,153.018686,153.015881,503.410156,201.177521,14.557462,5.817565,C7H5O4,153.01878363943
2,215.065487,215.074365,685.942200,392.497070,21.509676,12.307987,C13H11O3,215.07081921324
3,243.066303,243.065204,322.632690,173.600006,10.416106,5.604621,C14H11O4,243.06573383281
4,271.060642,271.060542,226059.375000,145707.937500,7490.086856,4827.780367,C15H11O5,271.06064845238
